In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats as stats
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm



In [ ]:
train_file = pd.read_csv('iot-datasets/train.csv')
test_file = pd.read_csv('iot-datasets/test.csv')
severity_type = pd.read_csv('iot-datasets/severity_type.csv')
event_type = pd.read_csv('iot-datasets/event_type.csv')
log_feature = pd.read_csv('iot-datasets/log_feature.csv')
resource_type = pd.read_csv('iot-datasets/resource_type.csv')
volume = log_feature[['id','volume']]
log_feature = log_feature.drop('volume', axis=1)

location = train_file[['id','location']]
train_file = train_file.drop('location',axis=1)


In [ ]:
print(f"severity_type shape:{severity_type.shape}" )
print(f"event_type shape:{event_type.shape}" )
print(f"log_feature shape:{log_feature.shape}" )
print(f"resource_type shape:{resource_type.shape}" )
print(f"volume shape:{volume.shape}" )
print(f"location shape:{location.shape}" )
print(f"train shape:{train_file.shape}" )



In [ ]:
severity_type.drop_duplicates(keep='first')
event_type.drop_duplicates(keep='first')
resource_type.drop_duplicates(keep='first')
log_feature.drop_duplicates(keep='first')
volume.drop_duplicates(keep='first')

print(f"severity_type shape:{severity_type.shape}" )
print(f"event_type shape:{event_type.shape}" )
print(f"log_feature shape:{log_feature.shape}" )
print(f"resource_type shape:{resource_type.shape}" )
print(f"volume shape:{volume.shape}" )



In [ ]:
severity_type = severity_type.pivot_table(index='id', columns='severity_type', aggfunc='size')
severity_type.reset_index(inplace=True)

event_type = event_type.pivot_table(index='id', columns='event_type', aggfunc='size')
event_type.reset_index(inplace=True)

resource_type = resource_type.pivot_table(index='id', columns='resource_type', aggfunc='size')
resource_type.reset_index(inplace=True)

log_feature = log_feature.pivot_table(index='id', columns='log_feature', aggfunc='size')
log_feature.reset_index(inplace=True)

location = location.pivot_table(index='id', columns='location', aggfunc='size')
location.reset_index(inplace=True)

prefix = 'Volume '
volume['volume'] = volume['volume'].apply(lambda x: prefix + str(x))
volume = volume.pivot_table(index='id', columns='volume', aggfunc='size')
volume.reset_index(inplace=True)





In [ ]:
volume


In [ ]:
df = pd.merge(train_file,severity_type,on='id',how='left')
df = pd.merge(df,event_type,on='id',how='left')
df = pd.merge(df,resource_type,on='id',how='left')
df = pd.merge(df,log_feature,on='id',how='left')
df = pd.merge(df,volume,on='id',how='left')
df_with_location = pd.merge(df,location,on='id',how='left')
df_without_location = df



In [ ]:
df = df_with_location
# df = df_without_location


In [ ]:
df.isnull().sum()


In [ ]:
df.fillna(0, inplace=True)
df.isnull().sum()


In [ ]:
df.shape


In [ ]:
df


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for column in df.columns:
    if column != 'id' or column!='fault_severity':
        df[column] = label_encoder.fit_transform(df[column])


In [ ]:
df


In [ ]:
df.dtypes


## Split dataset

In [ ]:
y = df['fault_severity']
X = df.drop(['fault_severity','id'], axis=1)
print("Number of examples: " + str(X.shape[0]))
print("\nNumber of Features:" + str(X.shape[1]))


In [ ]:
X_train,X_test, y_train,y_test =  train_test_split(X,y, test_size=0.33,random_state=1234)


## Logistic regression

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr_model.fit(X_train, y_train)
lr_y_pred = lr_model.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
lr_accuracy = accuracy_score(y_test, lr_y_pred)
print('Accuracy score of Logistic regression: ' + str(lr_accuracy))


## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=3)

knn_model.fit(X_train, y_train)

knn_y_pred = knn_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_y_pred)


In [ ]:
print('Accuracy score of KNN: ' + str(knn_accuracy))


## Decision Tree

In [ ]:
param_grid = {'max_depth':[4,8],'min_samples_leaf':[25,50]}
print('Running grid search')
dt_regressor = DecisionTreeClassifier()
dt_grid = GridSearchCV(dt_regressor,param_grid,cv=3, scoring='neg_root_mean_squared_error')
dt_grid_search = dt_grid.fit(X_train,y_train)
print('finished grid search')


#get the best parameters for decision tree
dt_best_params= dt_grid_search.best_params_
print(f"best parameters for decision tree model: {dt_best_params}")
dt_model = DecisionTreeClassifier(max_depth=dt_best_params['max_depth'], min_samples_leaf=dt_best_params['min_samples_leaf'])
dt_model.fit(X_train, y_train)
y_dt_pred =dt_model.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_dt_pred)




In [ ]:
print("Accuracy of Decision Tree:", dt_accuracy)


## Random Forest

In [ ]:
param_grid = {'max_depth':[4,8,10,20,32],'n_estimators':[25,50,100,200,300]}
print('Running grid search')
rf_model = RandomForestClassifier()
rf_grid = GridSearchCV(rf_model,param_grid,cv=3, scoring='neg_root_mean_squared_error')
rf_grid_search = rf_grid.fit(X_train,y_train)
print('finished grid search')

rf_best_params= rf_grid_search.best_params_
print(f"best parameters for rf model: {rf_best_params}")
rf_model = RandomForestClassifier(max_depth=rf_best_params['max_depth'],n_estimators = rf_best_params['n_estimators'])
rf_model.fit(X_train, y_train)
y_rf_pred=rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_rf_pred)


In [ ]:
print("Accuracy of Random Forest:", rf_accuracy)


## GBDT

In [ ]:

param_grid = {'max_depth':[4,8,10],'n_estimators':[25,50,100,200,300]}
print('Running grid search')
gbdt_model = GradientBoostingClassifier()
GBDT_grid = GridSearchCV(gbdt_model,param_grid,cv=3, scoring='neg_root_mean_squared_error')
GBDT_grid_search = GBDT_grid.fit(X_train,y_train)
print('finished grid search')


GBDT_best_params= GBDT_grid_search.best_params_
print(f"best parameters for GBDT model: {GBDT_best_params}")

gbdt_model = GradientBoostingClassifier(max_depth=GBDT_best_params['max_depth'], n_estimators=GBDT_best_params['n_estimators'])
gbdt_model.fit(X_train,y_train)
y_gbdt_pred =  gbdt_model.predict(X_test)


In [ ]:
gbdt_accuracy = accuracy_score(y_test, y_gbdt_pred)
print("Accuracy:", gbdt_accuracy)


## Naive Bayes

In [ ]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_y_pred = nb_model.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_y_pred)
print("Accuracy:", nb_accuracy)


## SVM

In [ ]:
svm_model_linear = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train, y_train)
svm_y_pred = svm_model_linear.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_y_pred)


In [ ]:
print("Accuracy of SVM:", svm_accuracy)


In [ ]:
feature_imp = gbdt_model.feature_importances_
df_features = pd.DataFrame({'name':X_train.columns.values, 'imp':feature_imp})
df_sorted =df_features.sort_values(by=['imp'],  ascending=False)
top_five = df_sorted.iloc[:6]['name'].tolist()
print('Top five features: {0}'.format(top_five))


In [ ]:
fig, ax = plt.subplots()
ax.bar(np.arange(6), sorted(rf_model.feature_importances_, reverse=True)[:6], width = 0.35)
ax.set_xticks(np.arange(6))
ax.set_xticklabels(top_five, rotation = 90)

for i, v in enumerate(sorted(rf_model.feature_importances_, reverse=True)[:6]):
    plt.text(i, v, f"{v:.2f}", ha='center', va='bottom')

plt.title('Feature importance from GBDT')
ax.set_ylabel('Normalized importance')


In [ ]:
accuracy_results = [lr_accuracy, knn_accuracy, dt_accuracy, rf_accuracy, gbdt_accuracy, nb_accuracy, svm_accuracy]
labels = ['LR', 'KNN', 'DT', 'RF', 'GBDT', 'NB', 'SVM']
label_accuracy_pairs = list(zip(labels, accuracy_results))
sorted_label_accuracy_pairs = sorted(label_accuracy_pairs, key=lambda x: x[1], reverse=True)
accuracy_results = [pair[1] for pair in sorted_label_accuracy_pairs]

labels = [pair[0] for pair in sorted_label_accuracy_pairs]

rg= np.arange(7)
width = 0.35
plt.bar(rg, accuracy_results, width, label="Accuracy")
plt.xticks(rg + width/2, labels)
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.ylim([0,1])

for i, v in enumerate(accuracy_results):
    plt.text(i, v, f"{v:.2f}", ha='center', va='bottom')


plt.title('Model Performance')
plt.legend(loc='upper left', ncol=2)
plt.show()
